title: Statistics behind upfront payment testing        
author: Fabio Schmidt-Fischbach   
date: 2021-07-05  
region: EU   
link: https://docs.google.com/presentation/d/148z3P03Nr_l2qpP8w4SvsIhoi1CN-OyoQAz6EIEBcqU/edit?usp=sharing     
tags: memberships, xsell, discount, upfront payments, upfront, premium, retention, monetization, bayesian, experimentation   

summary: "This proposes a way to run experiments in the context of the upfront payment functionality. The key challenge is to quantify the uncertainty around a non-binary outcome (revenue per user) in a Bayesian way. We do so by modelling revenue per user as a random variable composed by a "product selection" (dirichlet), a "price" and a retention (beta) component."


In [3]:
import pandas as pd
import scipy as sc
import altair as alt

## What is our north star metric? 

We model expected premium fee revenue within 12 months per user as our ultimate success metric. 

## Why is that hard? 

For users that don't choose to pay upfront 12 months in advance, we actually don't know how much they'll pay us. How can we compare these user groups? 

## How does our experiment look like? 

- Control group: only gets monthly billing options 
- Variant group(s): different design, differnt payment options (e.g. we vary frequency of payment and or discounts attached to different payment plans) 

## Modelling revenue per user as (x-sell) x (% that pay)

Revenue per user consists of two levers

- x-sell e.g. what % of people pick which plan and tier 
- retention e.g. what % of people that picked each plan also pay their bill. 

We believe that upfront payments can influence revenue per user through both levers - we should hence try to capture both effects as well as possible. 























Let 
- $ i \in \{1,2,3...\}$ be the index for the variants we want to compare.
- $ j$ be a particular payment plan in variant i which we can attach a front-loaded nominal price tag $p_{i,j}$. For the monthly smart tier $p_{control,smart-monthly}$ this would be $12 \times 4.90 = 58.80$. 
- let $f(i,j)$ be a pmf that summarizes how likely a user is to choose j if being presented variant i. If in variant-A 10% of users pick the monthly smart plan, then f(variant_a, monthly-smart)=10%. 
- let $d(i,j) \in [0,1] $ be a random variable that discounts the nominal price $p_{i,j}$ to account for churn. If $E(d(variant_a, smart-monthly)) = 50\%$ then we actually don't expect to collect 58.80 but 29.4 from each user that makes that choice.

Then the expected revenue per user $r_{i}$ from a variant can be summarized by 

$ r_{i} = \sum_{j} f_{i,j}(\theta_{i})) \times p_{i,j} \times d_{i,j}(\delta_{i})$

This sum consists of two stochastic elements (x-sell and discount factors) that we need to estimate and one vector of scalars $ p_{i,j} $ that we already know. The remaining challenge is to derive posterior distribution for the x-sell and the discount factor component of our formula. 





## How to model x-sell choices

We usually model events in AB tests that can only have two outcomes : convert or not. In our x-sell experiment, there are many more different possibilities. Imagine, we present 3 premium tiers and 2 different payment plans for each (monthly, yearly). In this scenario, a user can pick $3 \times 2$ + 1 ("not-premium") = 7 different things.

We hence model a user's choices $f_{i,j}(\theta_{i})$ in variant $i$ as a draw from a multinomial distribution (https://en.wikipedia.org/wiki/Multinomial_distribution). This is a natural choice and also handy since the multinomial distribution has a easy-to-handle-conjugate prior (Dirichlet) for the use in Bayesian statistics. 

Bayesian inference of the Dirichlet's parameters (e.g. the x-sell probabilities of each plan $j$) is standard. 

## How to model discount factors 

Here, we need to cut some corners since we won't be able to wait 12 months until actual payments have realized. I propose to model a discount factor associated with variant $i$ and plan $j$ as a Bernoulli process (beta prior). Think of a bernoulli RV to model the probability of an (un-) fair coin showing heads at a coin. The expected value of this flip is the discount rate we want to estimate. 

- For the monthly payment, I propose to estimate the discount rate wit historicals and fix them via the priors of the Beta prior to the Bernoulli (basically, we'd not add evidence to the prior and sample from it directly). 
- For the upfront payments, I propose to grant a user 14 days to pay. All users that pay within that timeframe would be considered a conversion, everyone else not. This is what I'd use to feed as evidence into the prior. For upfront plans that require more than one payment $t$, I propose to compute the discount rate:

$ d_{i,j}(\delta_{i}) =\delta_{i,j}^{t}$.

As an example, if our posterior for the first payment of our bi-annual plan centers around 70%, then we'd use $0.7^{2}=0.49$ as the discount factor.


Modelling this as a bernoulli process makes inference fairly easy (since we know how to handle a beta prior). 

## Sampling from the posteriors

We can use the loss-function approach to compare the posteriors. To do so, we sample x-sell weights from f(i,j) and discount factors d(i,j) from the posteriors and plug it into 

$ r_{i}(\theta_{i}, \delta_{i}) = \sum_{j} f_{i,j}(\theta_{i}) \times p_{i,j} \times d_{i,j}(\delta_{i})$

for each variant i. 

This returns one sample for revenue for each variant. We repeat this M times and then compare whether $r_{A} > r_{B}$. Prob(A>B) is # of times $(r_{A} > r_{B})/ M$. 

Compare this paper for a similar analysis. 

https://cdn2.hubspot.net/hubfs/310840/VWO_SmartStats_technical_whitepaper.pdf

## Deriving priors for monthly options 

In [2]:
df = pd.read_csv("data.csv")

price = {"Metal": 16.90, "Smart": 4.90, "You": 9.90}

df["price"] = df["tier"].map(price)

df = df.groupby(["payment_no", "tier", "price"]).agg("sum").reset_index()
df["payment"] = 100 * df["paid"] / df["users"]

alt.Chart(df).mark_line().encode(
    x=alt.X("payment_no:N"), y=alt.Y("payment:Q"), color="tier:N"
).properties(width=500, height=500)

alt.Chart(...)

In [3]:
df = pd.read_csv("data.csv")

price = {"Metal": 16.90, "Smart": 4.90, "You": 9.90}

df["price"] = df["tier"].map(price)

df = df.groupby(["payment_no", "tier", "price"]).agg("sum").reset_index()
df["expected_rev"] = df["price"] * df["paid"] / df["users"]

alt.Chart(df).mark_line().encode(
    x=alt.X("payment_no:N"), y=alt.Y("expected_rev:Q"), color="tier:N"
).properties(width=500, height=500)

alt.Chart(...)

In [4]:
df = pd.read_csv("data.csv")

price = {"Metal": 16.90, "Smart": 4.90, "You": 9.90}

df["price"] = df["tier"].map(price)

df = df.groupby(["payment_no", "tier", "price"]).agg("sum").reset_index()
df["expected_rev"] = df["price"] * df["paid"] / df["users"]

df = df.groupby(["tier"])["expected_rev"].agg("sum").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("tier:N"),
    y=alt.Y("expected_rev:Q"),
).properties(width=500, height=500)

alt.Chart(...)

In [10]:
16.90 * 12

202.79999999999998

In [33]:
def compute_return(monthly_price, conversions, charges):
    """
    Take base price and discount it. Return expected revenue
    """
    charge_price = monthly_price * 12 / charges
    total = 0
    for i in range(1, charges + 1):
        total += charge_price * (conversions**i)
    return total


prices = list(range(10, 20, 1))

df = pd.DataFrame(prices, columns=["prices"])
df["conversion"] = 0.7
df["charges"] = 2

df["return"] = df.apply(
    lambda x: compute_return(x["prices"], x["conversion"], int(x["charges"])), axis=1
)

df["difference_to_status_quo"] = df["return"] - 90

alt.Chart(df).mark_bar().encode(
    x=alt.X("prices:Q", axis=alt.Axis(title="Equivalent monthly price")),
    y=alt.Y(
        "difference_to_status_quo:Q",
        axis=alt.Axis(title="Excess profit versus current monthly baseline"),
    ),
).properties(width=500, height=500, title="Bi-annual/70% payment on each charge")

0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994
0.7
0.48999999999999994


alt.Chart(...)

In [34]:
df.head(10)

,prices,conversion,charges,return,difference_to_status_quo
0,10,0.7,2,71.40,-18.60
1,11,0.7,2,78.54,-11.46
2,12,0.7,2,85.68,-4.32
3,13,0.7,2,92.82,2.82
4,14,0.7,2,99.96,9.96
5,15,0.7,2,107.10,17.10
6,16,0.7,2,114.24,24.24
7,17,0.7,2,121.38,31.38
8,18,0.7,2,128.52,38.52
9,19,0.7,2,135.66,45.66


In [48]:
from scipy.stats import dirichlet

# compute expected revenue for each option

# get evidence.
conversions = {"metal": 10, "standard/noconversion": 120, "you": 30, "smart": 50}

dirichlet.rvs(list(conversions.values()), size=1000)

array([[0.0588996 , 0.55994855, 0.1340318 , 0.24712005],
       [0.06035535, 0.5428976 , 0.16152879, 0.23521826],
       [0.0483023 , 0.58834666, 0.10138335, 0.26196769],
       ...,
       [0.02029407, 0.52752466, 0.2000446 , 0.25213667],
       [0.04035482, 0.51428824, 0.16108742, 0.28426952],
       [0.04375469, 0.59452114, 0.11318977, 0.2485344 ]])